In [1]:
import numpy as np

PATTERN_TO_DEBUG = 0

def toDec(hexstr,BIT):
    msb4bits = hexstr[0]
    n = int(msb4bits, 16)
    if n >= 8:
        p = -1*pow(2,BIT-1)
        addend = int(str(n-8) + hexstr[1:], 16)
        return str( p + addend)
    else:
        return str(int(hexstr, 16))

def int2hex(num,bit_width):
    if num<0:
        num = (1 << bit_width) + num
    hex_string = hex(num)[2:]
    return '{:0>{width}}'.format(hex_string,width = (bit_width+3)//4)

NEGATE = {'1': '0', '0': '1'}
def negate(value):
    return ''.join(NEGATE[x] for x in value)

def twocomplement(n, size_in_bits):
    # print(n)
    number = int(n)
    if number < 0:
        return negate(bin(abs(number) - 1)[2:]).rjust(size_in_bits, '1')
    else:
        return bin(number)[2:].rjust(size_in_bits, '0')

NEGATE = {'1': '0', '0': '1'}
def negate(value):
    return ''.join(NEGATE[x] for x in value)

def twocomplement(n, size_in_bits):
    # print(n)
    number = int(n)
    if number < 0:
        return negate(bin(abs(number) - 1)[2:]).rjust(size_in_bits, '1')
    else:
        return bin(number)[2:].rjust(size_in_bits, '0')


In [2]:
with open('input.txt', 'r') as file:
  file_in = file.readlines()

NUM = int(file_in[0])

# print(file_in)
for i in range(NUM):
  img_16 = []
  kernal_16 = []
  matrix_idx = []
  mode = []

  # MATRIX SIZE
  matrix_in_idx = int(file_in[1 + 65 * i + 0])

  if matrix_in_idx == 0:
     matrix_size = 8
  elif matrix_in_idx == 1:
     matrix_size = 16
  else:
     matrix_size = 32

  for j in range(16):
    img_in = np.array([int(toDec(val,8)) for val in file_in[1 + 65 * i + (1+j)].split()])
    img_16.append(np.reshape(img_in,(matrix_size,matrix_size)))

  for k in range(16):
    kernal_in = np.array([int(toDec(val,8)) for val in file_in[1 + 65 * i + (17+k)].split()])
    kernal_16.append(np.reshape(kernal_in,(5,5)))


  for idx in range(16):
    mode.append(int(file_in[1 + 65 * i + (33+idx*2)])) #0 2 4
    matrix_idx.append(np.array([int(val) for val in file_in[1 + 65 * i + (34+idx*2)].split()]))# 1 3 5

  if i == PATTERN_TO_DEBUG:
    break

In [3]:
mode[0]

1

In [4]:
matrix_idx[0]

array([8, 0])

In [5]:
img_16[0]

array([[2, 1, 1, 1, 1, 1, 1, 2],
       [1, 1, 1, 1, 2, 2, 2, 1],
       [1, 1, 1, 1, 2, 1, 1, 2],
       [1, 1, 1, 2, 1, 2, 2, 2],
       [2, 1, 1, 2, 1, 1, 1, 2],
       [1, 2, 2, 1, 1, 1, 2, 1],
       [1, 1, 1, 1, 2, 1, 1, 2],
       [2, 1, 2, 2, 1, 1, 2, 1]])

In [6]:
kernal_16[15]

array([[2, 1, 1, 1, 1],
       [1, 1, 2, 2, 2],
       [1, 2, 2, 2, 1],
       [1, 1, 2, 2, 2],
       [2, 1, 1, 1, 1]])

In [7]:
matrix_idx[0]

array([8, 0])

In [8]:
matrix_size

8

## Convolution

In [9]:
img = img_16[0]
kernal = kernal_16[0]

convoluted_results = np.zeros((matrix_size-4,matrix_size-4))
for i in range(matrix_size-4):
    for j in range(matrix_size-4):
        conv = 0
        for k in range(5):
            for l in range(5):
                conv += kernal[k,l] * img[i+k,j+l]

        convoluted_results[i,j] = conv

In [10]:
convoluted_results

array([[46., 43., 48., 55.],
       [47., 50., 50., 52.],
       [45., 47., 45., 53.],
       [49., 48., 51., 51.]])

## Maxpooling

In [11]:
convoluted_img_width = len(convoluted_results)

In [12]:
max_pooling_results = np.zeros((convoluted_img_width//2,convoluted_img_width//2))

for i in range(0,convoluted_img_width,2):
    for j in range(0,convoluted_img_width,2):
        max_pooling_results[i//2,j//2] = max(convoluted_results[i,j],convoluted_results[i+1,j],\
                                                convoluted_results[i+1,j+1],convoluted_results[i,j+1])

In [13]:
max_pooling_results

array([[50., 55.],
       [49., 53.]])

### Merged Convolution and max pooling

In [14]:
max_pooling_results = np.zeros((convoluted_img_width//2,convoluted_img_width//2))
for img_yptr in range(0,matrix_size-4,2):
    for img_xptr in range(0,matrix_size-4,2):
        print(img_yptr)

        temp_max = -999999
        # 4 times convolution to quickly get values needed for max pooling
        for mp_window_y in range(2):
            for mp_window_x in range(2):
                conv = 0
                idx_y = img_yptr+mp_window_y
                idx_x = img_xptr+mp_window_x
                for k_yptr in range(5):
                    # Uses High level modeling for fast MAC, img_x_cord[x] and kernal_x_cord[x]
                    for k_xptr in range(5): # Unrolling the x_ptr
                        conv += kernal[k_yptr,k_xptr] * img[idx_y+k_yptr,idx_x+k_xptr]

                if (mp_window_y == 0 and mp_window_x == 0) or conv > temp_max:
                    temp_max = conv

        max_pooling_results[img_yptr//2,img_xptr//2] = temp_max

In [15]:
max_pooling_results

array([[50., 55.],
       [49., 53.]])

In [16]:
max_pooling_results = np.zeros((convoluted_img_width//2,convoluted_img_width//2))
for img_yptr in range(0,matrix_size-4,2):
    for img_xptr in range(0,matrix_size-4,2):
        temp_max = -999999
        # 4 times convolution to quickly get values needed for max pooling
        for mp_window_y in range(2):
            for mp_window_x in range(2):
                conv = 0
                idx_y = img_yptr+mp_window_y
                idx_x = img_xptr+mp_window_x
                for k_yptr in range(5):
                    conv += kernal[k_yptr,0] * img[idx_y+k_yptr,idx_x+0]
                    conv += kernal[k_yptr,1] * img[idx_y+k_yptr,idx_x+1]
                    conv += kernal[k_yptr,2] * img[idx_y+k_yptr,idx_x+2]
                    conv += kernal[k_yptr,3] * img[idx_y+k_yptr,idx_x+3]
                    conv += kernal[k_yptr,4] * img[idx_y+k_yptr,idx_x+4]

                if (mp_window_y == 0 and mp_window_x == 0) or conv > temp_max:
                    temp_max = conv

        max_pooling_results[img_yptr//2,img_xptr//2] = temp_max

In [17]:
max_pooling_results

array([[50., 55.],
       [49., 53.]])

### Transposed convolution

In [18]:
trans_convoluted_results = np.zeros((matrix_size+4,matrix_size+4))
for i in range(matrix_size):
    for j in range(matrix_size):
        for k in range(5):
            for l in range(5):
                conv = kernal[k,l] * img[i,j]
                trans_convoluted_results[i+k,j+l] += conv

In [19]:
trans_convoluted_results

array([[ 2.,  3.,  4.,  5.,  8.,  6.,  6.,  7.,  6.,  5.,  4.,  4.],
       [ 5.,  6., 10., 11., 15., 15., 16., 18., 15., 13.,  8.,  4.],
       [ 5.,  8., 14., 16., 25., 24., 26., 27., 24., 18., 11.,  9.],
       [ 8., 11., 19., 25., 33., 33., 37., 42., 33., 29., 19., 10.],
       [11., 17., 24., 35., 44., 45., 48., 54., 43., 33., 21., 13.],
       [10., 16., 26., 35., 45., 49., 50., 55., 40., 35., 23., 11.],
       [ 9., 17., 27., 36., 46., 47., 47., 52., 42., 32., 22., 13.],
       [11., 17., 28., 41., 49., 47., 53., 52., 39., 32., 24., 10.],
       [11., 17., 27., 36., 42., 38., 41., 38., 32., 24., 16.,  8.],
       [ 6., 12., 21., 25., 33., 30., 32., 32., 23., 19., 14.,  5.],
       [ 6.,  8., 14., 19., 22., 22., 22., 22., 17., 12.,  9.,  3.],
       [ 4.,  6.,  8., 13., 12., 11., 13., 11.,  7.,  6.,  4.,  1.]])

## Transpoed convolution with zero-padding

In [20]:
new_trans_convoluted_results = np.zeros((matrix_size+4,matrix_size+4))
lower_bound0 = matrix_size + 4
lower_bound1 = matrix_size + 5
lower_bound2 = matrix_size + 6
lower_bound3 = matrix_size + 7

for img_ytr in range(matrix_size+4):
    for img_xptr in range(matrix_size+4):
        conv = 0
        for k_yptr in range(5):
            # Uses High level always@* , need idx_x[x]
            for k_xptr in range(5):
                idx_y = img_ytr + k_yptr
                idx_x = img_xptr + k_xptr
                if idx_y == 0 or idx_y == 1 or idx_y == 2 or idx_y == 3 or idx_x == 0 or idx_x == 1 \
                    or idx_x == 2 or idx_x == 3:
                        conv += 0
                elif idx_y == lower_bound0 or idx_y == lower_bound1 or idx_y == lower_bound2 or idx_y == lower_bound3 \
                    or idx_x == lower_bound0 or idx_x == lower_bound1 or idx_x == lower_bound2 or idx_x == lower_bound3 :
                        conv += 0
                else:
                   # Address translation
                   conv += kernal[4-k_yptr,4-k_xptr] * img[idx_y-4,idx_x-4]

        new_trans_convoluted_results[img_ytr,img_xptr] = conv


In [21]:
new_trans_convoluted_results

array([[ 2.,  3.,  4.,  5.,  8.,  6.,  6.,  7.,  6.,  5.,  4.,  4.],
       [ 5.,  6., 10., 11., 15., 15., 16., 18., 15., 13.,  8.,  4.],
       [ 5.,  8., 14., 16., 25., 24., 26., 27., 24., 18., 11.,  9.],
       [ 8., 11., 19., 25., 33., 33., 37., 42., 33., 29., 19., 10.],
       [11., 17., 24., 35., 44., 45., 48., 54., 43., 33., 21., 13.],
       [10., 16., 26., 35., 45., 49., 50., 55., 40., 35., 23., 11.],
       [ 9., 17., 27., 36., 46., 47., 47., 52., 42., 32., 22., 13.],
       [11., 17., 28., 41., 49., 47., 53., 52., 39., 32., 24., 10.],
       [11., 17., 27., 36., 42., 38., 41., 38., 32., 24., 16.,  8.],
       [ 6., 12., 21., 25., 33., 30., 32., 32., 23., 19., 14.,  5.],
       [ 6.,  8., 14., 19., 22., 22., 22., 22., 17., 12.,  9.,  3.],
       [ 4.,  6.,  8., 13., 12., 11., 13., 11.,  7.,  6.,  4.,  1.]])

In [22]:
new_trans_convoluted_results = np.zeros((matrix_size+4,matrix_size+4))
lower_bound0 = matrix_size + 4
lower_bound1 = matrix_size + 5
lower_bound2 = matrix_size + 6
lower_bound3 = matrix_size + 7

for img_ytr in range(matrix_size+4):
    for img_xptr in range(matrix_size+4):
        conv = 0
        for k_yptr in range(5):
            # for k_xptr in range(5):
                idx_y = img_ytr + k_yptr
                idx_x_0 = img_xptr + 0
                idx_x_1 = img_xptr + 1
                idx_x_2 = img_xptr + 2
                idx_x_3 = img_xptr + 3
                idx_x_4 = img_xptr + 4
                # 0
                if idx_y == 0 or idx_y == 1 or idx_y == 2 or \
                    idx_y == 3 or idx_x_0 == 0 or idx_x_0 == 1 \
                    or idx_x_0 == 2 or idx_x_0 == 3:
                        conv += 0
                elif idx_y == lower_bound0 or idx_y == lower_bound1 or idx_y == lower_bound2 or idx_y == lower_bound3 \
                    or idx_x_0 == lower_bound0 or idx_x_0 == lower_bound1 or idx_x_0 == lower_bound2 or idx_x_0 == lower_bound3 :
                        conv += 0
                else:
                   conv += kernal[4-k_yptr,4] * img[idx_y-4,idx_x_0-4]
                # 1
                if idx_y == 0 or idx_y == 1 or idx_y == 2 or \
                    idx_y == 3 or idx_x_1 == 0 or idx_x_1 == 1 \
                    or idx_x_1 == 2 or idx_x_1 == 3:
                        conv += 0
                elif idx_y == lower_bound0 or idx_y == lower_bound1 or idx_y == lower_bound2 or idx_y == lower_bound3 \
                    or idx_x_1 == lower_bound0 or idx_x_1 == lower_bound1 or idx_x_1 == lower_bound2 or idx_x_1 == lower_bound3 :
                        conv += 0
                else:
                   conv += kernal[4-k_yptr,3] * img[idx_y-4,idx_x_1-4]
                # 2
                if idx_y == 0 or idx_y == 1 or idx_y == 2 or \
                    idx_y == 3 or idx_x_2 == 0 or idx_x_2 == 1 \
                    or idx_x_2 == 2 or idx_x_2 == 3:
                        conv += 0
                elif idx_y == lower_bound0 or idx_y == lower_bound1 or idx_y == lower_bound2 or idx_y == lower_bound3 \
                    or idx_x_2 == lower_bound0 or idx_x_2 == lower_bound1 or idx_x_2 == lower_bound2 or idx_x_2 == lower_bound3 :
                        conv += 0
                else:
                   conv += kernal[4-k_yptr,2] * img[idx_y-4,idx_x_2-4]
                # 3
                if idx_y == 0 or idx_y == 1 or idx_y == 2 or \
                    idx_y == 3 or idx_x_3 == 0 or idx_x_3 == 1 \
                    or idx_x_3 == 2 or idx_x_3 == 3:
                        conv += 0
                elif idx_y == lower_bound0 or idx_y == lower_bound1 or idx_y == lower_bound2 or idx_y == lower_bound3 \
                    or idx_x_3 == lower_bound0 or idx_x_3 == lower_bound1 or idx_x_3 == lower_bound2 or idx_x_3 == lower_bound3 :
                        conv += 0
                else:
                   conv += kernal[4-k_yptr,1] * img[idx_y-4,idx_x_3-4]

                # 4
                if idx_y == 0 or idx_y == 1 or idx_y == 2 or \
                    idx_y == 3 or idx_x_4 == 0 or idx_x_4 == 1 \
                    or idx_x_4 == 2 or idx_x_4 == 3:
                        conv += 0
                elif idx_y == lower_bound0 or idx_y == lower_bound1 or idx_y == lower_bound2 or idx_y == lower_bound3 \
                    or idx_x_4 == lower_bound0 or idx_x_4 == lower_bound1 or idx_x_4 == lower_bound2 or idx_x_4 == lower_bound3 :
                        conv += 0
                else:
                   conv += kernal[4-k_yptr,0] * img[idx_y-4,idx_x_4-4]

        new_trans_convoluted_results[img_ytr,img_xptr] = conv


In [23]:
new_trans_convoluted_results

array([[ 2.,  3.,  4.,  5.,  8.,  6.,  6.,  7.,  6.,  5.,  4.,  4.],
       [ 5.,  6., 10., 11., 15., 15., 16., 18., 15., 13.,  8.,  4.],
       [ 5.,  8., 14., 16., 25., 24., 26., 27., 24., 18., 11.,  9.],
       [ 8., 11., 19., 25., 33., 33., 37., 42., 33., 29., 19., 10.],
       [11., 17., 24., 35., 44., 45., 48., 54., 43., 33., 21., 13.],
       [10., 16., 26., 35., 45., 49., 50., 55., 40., 35., 23., 11.],
       [ 9., 17., 27., 36., 46., 47., 47., 52., 42., 32., 22., 13.],
       [11., 17., 28., 41., 49., 47., 53., 52., 39., 32., 24., 10.],
       [11., 17., 27., 36., 42., 38., 41., 38., 32., 24., 16.,  8.],
       [ 6., 12., 21., 25., 33., 30., 32., 32., 23., 19., 14.,  5.],
       [ 6.,  8., 14., 19., 22., 22., 22., 22., 17., 12.,  9.,  3.],
       [ 4.,  6.,  8., 13., 12., 11., 13., 11.,  7.,  6.,  4.,  1.]])

#### Convert results to LSB binary representation

In [24]:
twocomplement(-1,8)

'11111111'

In [25]:
vf = np.vectorize(twocomplement)
bits_result = vf(trans_convoluted_results,20)

In [26]:
bits_result[0][0]

'00000000000000000010'

In [27]:
result = bits_result[0][0]

In [28]:
result = result[::-1]

In [29]:
result

'01000000000000000000'